# Fetching pit-stop data for Formula 1 season

### The Data is obtained using **FastF1** library

In [1]:
import fastf1
import pandas as pd

In [2]:
fastf1.Cache.enable_cache('cache')

In [3]:
season = 2024
races = fastf1.get_event_schedule(season)

In [79]:
pit_stop_data = []

for round_number in races.index:
    if round_number == 0:
        continue
    session = fastf1.get_session(season, round_number, 'R')
    session.load(laps=True)

    results = session.results[['Abbreviation', 'TeamName', 'DriverNumber', 'GridPosition', 'ClassifiedPosition']].set_index('DriverNumber')
    for driver_number in session.drivers:
        driver_abbrev = results.loc[driver_number, 'Abbreviation']
        team = results.loc[driver_number, 'TeamName']

        grid_position = results.loc[driver_number, 'GridPosition']
        position = results.loc[driver_number, 'ClassifiedPosition']

        driver_laps = session.laps[session.laps['Driver'] == driver_abbrev]

        pit_in_laps = driver_laps[driver_laps['PitInTime'].notna()]
        pit_out_laps = driver_laps[driver_laps['PitOutTime'].notna()]

        for _, in_lap in pit_in_laps.iterrows():
            out_lap = pit_out_laps[pit_out_laps['LapNumber'] == in_lap['LapNumber'] + 1]

            if not out_lap.empty:
                out_lap = out_lap.iloc[0]  # Take the first match

                # Compute pit stop duration
                pit_duration = (out_lap['PitOutTime'] - in_lap['PitInTime']).total_seconds()

                pit_stop_data.append({
                    'Race': session.event['EventName'],
                    'Driver': driver_abbrev,
                    'Team': team,
                    'Grid Position': grid_position,
                    'Position': position,
                    'Lap': in_lap['LapNumber'],
                    'Tyre Before': in_lap['Compound'],
                    'Tyre After': out_lap['Compound'],
                    'Pit Duration (s)': round(pit_duration, 3)  # Round to milliseconds
                })

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']
core           INFO 	Loading data for Saudi Arabian Grand Prix

In [80]:
df = pd.DataFrame(pit_stop_data)
df

,Race,Driver,Team,Grid Position,Position,Lap,Tyre Before,Tyre After,Pit Duration (s)
0,Bahrain Grand Prix,VER,Red Bull Racing,1.0,1,17.0,SOFT,HARD,25.088
1,Bahrain Grand Prix,VER,Red Bull Racing,1.0,1,37.0,HARD,SOFT,24.218
2,Bahrain Grand Prix,PER,Red Bull Racing,5.0,2,12.0,SOFT,HARD,24.493
3,Bahrain Grand Prix,PER,Red Bull Racing,5.0,2,36.0,HARD,SOFT,24.423
4,Bahrain Grand Prix,SAI,Ferrari,4.0,3,14.0,SOFT,HARD,24.580
...,...,...,...,...,...,...,...,...,...
819,Abu Dhabi Grand Prix,LAW,RB,12.0,17,23.0,MEDIUM,HARD,23.456
820,Abu Dhabi Grand Prix,LAW,RB,12.0,17,24.0,HARD,HARD,22.486
821,Abu Dhabi Grand Prix,LAW,RB,12.0,17,29.0,HARD,HARD,32.260
822,Abu Dhabi Grand Prix,BOT,Kick Sauber,9.0,R,6.0,MEDIUM,HARD,41.098


In [83]:
df.to_csv('pit_stop_data.csv', index=False)

In [12]:
session = fastf1.get_session(season, 1, 'R')
session.load(laps=True)

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']
